In [2]:
from bs4 import BeautifulSoup
import requests
import geemap
import ee
import pandas as pd
import sched, time
from twisted.internet import task, reactor

In [3]:
s = sched.scheduler(time.time, time.sleep)
Map = geemap.Map()

In [4]:
def doWork():
    url = "https://waterdata.usgs.gov/mi/nwis/current/?type=flow&group_key=county_cd&site_no_name_select=siteno"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.select('table')[1]
    table_body_r = table.find('tbody').find_all("tr")
    l =[]
    for tr in table_body_r:
        td = tr.find_all('td')
        row = [str(tr.get_text()).strip() for tr in td]
        l.append(row)
    l2 = [l[98],l[99]]
    columns = table.find("thead").find_all("th")
    column_names = [c.text for c in columns]
    df2= pd.DataFrame(l2, columns = column_names)
    df2.columns = df2.columns.str.replace('\n', '')
    df2.insert(2, "latitude", ['47.08416667', '47.22861111' ], True)
    df2.insert(3, "longitude", ['-88.55055556', '-88.38527778' ], True)
    df2.to_csv('file_name1.csv', header = 'TRUE')
    in_csv = 'file_name1.csv'
    Map.add_xy_data(in_csv, x="longitude", y="latitude", layer_name="csv to ee", to_ee=True)
    fc = geemap.csv_to_ee(in_csv)
    Map.addLayer(fc, {}, 'csv to ee 2')
    

In [8]:
s.enter(5, 1, doWork, ())
s.run()

In [ ]:
doWork()

In [9]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…